In [2]:
import requests
import json
import re
import pdfkit
import json
import sys
import time
import pandas as pd
from user_agent import generate_user_agent
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from tqdm import tqdm
tqdm.pandas()

In [2]:
import pandas as pd
df = pd.read_parquet('data_with_embedding_v1.parquet')
df.head()

,ticker,filing_date,section,year_of_filing,company_name,context,embedding
0,AMZN,2010-01-29,Business,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nThis Annual Re...,"[0.064206064, -0.54433095, -0.36170813, 0.0565..."
1,AMZN,2010-01-29,Business,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nWe completed o...,"[0.19773945, -0.6633003, -0.32106644, -0.04229..."
2,AMZN,2010-01-29,Business,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nWe have organi...,"[0.18689841, -0.24920216, -0.38556015, 0.06467..."
3,AMZN,2010-01-29,Business,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nWe also manufa...,"[0.14794555, -0.4436285, -0.34766313, 0.061945..."
4,AMZN,2010-01-29,Business,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nWe operate cus...,"[0.06934562, -0.21145937, -0.36523664, 0.01908..."


In [3]:
df.tail()

,ticker,filing_date,section,year_of_filing,company_name,context,embedding
19198,NFLX,2020-01-29,"Exhibits, Financial Statement Schedules",2020,Netflix Inc,Company: Netflix Inc (NFLX)\nAmendment of Prov...,"[0.30167565, -0.035140604, -0.27240697, -0.065..."
19199,NFLX,2020-01-29,"Exhibits, Financial Statement Schedules",2020,Netflix Inc,Company: Netflix Inc (NFLX)\nThe Netherlands N...,"[0.11987999, -0.12105454, -0.33098242, -0.1603..."
19200,NFLX,2020-01-29,"Exhibits, Financial Statement Schedules",2020,Netflix Inc,Company: Netflix Inc (NFLX)\nDated: January 29...,"[0.118319005, 0.09442451, -0.39825603, -0.1186..."
19201,NFLX,2021-01-28,"Exhibits, Financial Statement Schedules",2021,Netflix Inc,Company: Netflix Inc (NFLX)\n(a)The following ...,"[0.14638504, -0.06706725, -0.40826216, -0.0354..."
19202,NFLX,2022-01-27,"Exhibits, Financial Statement Schedules",2022,Netflix Inc,Company: Netflix Inc (NFLX)\n(a)The following ...,"[0.14638504, -0.06706725, -0.40826216, -0.0354..."


In [39]:
def get_filing_id(content):
    # Generate a user agent string for Chrome
    user_agent = generate_user_agent(navigator='chrome')

    lookup_headers = {
        "referer": "https://sec-api.io",
        "user-agent": user_agent
    }

    # content['ticker'] = "NFLX"
    # date = 2022
    file_link = str()
    company_name = str()
    company_ticker = str()
    filing_id = str()
    data = {
            "query": {"query_string": {
                                        "query": "formType:\"10-K\" AND ticker:(" + content['ticker'] + ") AND filedAt:[" + str(
                                        content['year_of_filing']) + "-01-01 TO " + str(content['year_of_filing']+1) + "-01-01] AND documentFormatFiles.type:*21*"
                                        }
                    }, "sort": [{
                                    "filedAt": {"order": "desc"}
                                    }]}

    try:
        print(content['ticker'] + str(content['year_of_filing']))
        response = requests.post(
            "https://api.sec-api.io/", headers=lookup_headers, json=data, timeout=20)
        time.sleep(1)
    except requests.exceptions.Timeout:
        print('The request timed out')

    if response.status_code == 200:
        parsed_json = json.loads(response.text)
        hits = parsed_json['total']['value']
        if hits > 0:
            company_name = parsed_json['filings'][0]['companyName']
            company_ticker = parsed_json['filings'][0]['ticker']
            examine_link = parsed_json['filings'][0]['linkToFilingDetails']
            filing_id = parsed_json['filings'][0]['id']
            if examine_link.find('ix?doc=/') != -1:
                file_link = examine_link.replace('ix?doc=/', '')
            else:
                file_link = examine_link
        else:
            print('No hits')
    else:
        print(response.status_code)
    return filing_id

In [40]:
new_df = df.loc[:, ['ticker', 'year_of_filing']].drop_duplicates().reset_index(drop=True).copy()
new_df.head()

,ticker,year_of_filing
0,AMZN,2010
1,AMZN,2011
2,AMZN,2012
3,AMZN,2013
4,AMZN,2014


In [41]:
new_df['filing_id'] = new_df.progress_apply(get_filing_id, axis=1)

  0%|          | 0/69 [00:00<?, ?it/s]

AMZN2010


  3%|▎         | 2/69 [00:01<00:42,  1.59it/s]

AMZN2011


  4%|▍         | 3/69 [00:02<00:57,  1.14it/s]

...

 99%|█████████▊| 68/69 [01:22<00:01,  1.21s/it]

NFLX2021


100%|██████████| 69/69 [01:23<00:00,  1.22s/it]

NFLX2022


100%|██████████| 69/69 [01:24<00:00,  1.23s/it]


In [42]:
new_df[new_df['filing_id'] == '']

,ticker,year_of_filing,filing_id
18,GOOG,2021,
56,NFLX,2010,
57,NFLX,2011,
58,NFLX,2012,
59,NFLX,2013,


In [46]:
new_df.at[18, 'filing_id'] = 'e5b67aca9c746fbeb78a284c187c7efc'
new_df.at[56, 'filing_id'] = 'ffa98849e10e1046fc26f9649baf6f94'
new_df.at[57, 'filing_id'] = '606e270bc9cba63106fa3231f4ee31ae'
new_df.at[58, 'filing_id'] = '213bb461b86e95f40ca7e182492d0e2c'
new_df.at[59, 'filing_id'] = '73c550cbef01d1fba6a44ab484dff5db'

In [47]:
new_df[(new_df['ticker'] == 'NFLX')].drop_duplicates()

,ticker,year_of_filing,filing_id
56,NFLX,2010,ffa98849e10e1046fc26f9649baf6f94
57,NFLX,2011,606e270bc9cba63106fa3231f4ee31ae
58,NFLX,2012,213bb461b86e95f40ca7e182492d0e2c
59,NFLX,2013,73c550cbef01d1fba6a44ab484dff5db
60,NFLX,2014,7408333d4398fb377a440fc797eb9157
61,NFLX,2015,ac0212ec39ad81c404d11124cda9e072
62,NFLX,2016,5a98d5a867cc632d4840fdc9f5933fe9
63,NFLX,2017,4a40ccf944d9aaee880a3652439d586e
64,NFLX,2018,87f7af12437b83b9cae9c135e4b89181
65,NFLX,2019,690287c31d36511f8b8c3df884b0b1f5


In [50]:
# merging with the original dataframes
test_df = df.merge(new_df, on=['ticker', 'year_of_filing'], how='left')
test_df.head()

,ticker,filing_date,section,year_of_filing,company_name,context,embedding,filing_id
0,AMZN,2010-01-29,Business,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nThis Annual Re...,"[0.064206064, -0.54433095, -0.36170813, 0.0565...",de30fba5d4db4be30007c8119dd8451f
1,AMZN,2010-01-29,Business,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nWe completed o...,"[0.19773945, -0.6633003, -0.32106644, -0.04229...",de30fba5d4db4be30007c8119dd8451f
2,AMZN,2010-01-29,Business,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nWe have organi...,"[0.18689841, -0.24920216, -0.38556015, 0.06467...",de30fba5d4db4be30007c8119dd8451f
3,AMZN,2010-01-29,Business,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nWe also manufa...,"[0.14794555, -0.4436285, -0.34766313, 0.061945...",de30fba5d4db4be30007c8119dd8451f
4,AMZN,2010-01-29,Business,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nWe operate cus...,"[0.06934562, -0.21145937, -0.36523664, 0.01908...",de30fba5d4db4be30007c8119dd8451f


In [52]:
test_df.drop(columns=['section'], inplace=True)
test_df.head()

,ticker,filing_date,year_of_filing,company_name,context,embedding,filing_id
0,AMZN,2010-01-29,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nThis Annual Re...,"[0.064206064, -0.54433095, -0.36170813, 0.0565...",de30fba5d4db4be30007c8119dd8451f
1,AMZN,2010-01-29,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nWe completed o...,"[0.19773945, -0.6633003, -0.32106644, -0.04229...",de30fba5d4db4be30007c8119dd8451f
2,AMZN,2010-01-29,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nWe have organi...,"[0.18689841, -0.24920216, -0.38556015, 0.06467...",de30fba5d4db4be30007c8119dd8451f
3,AMZN,2010-01-29,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nWe also manufa...,"[0.14794555, -0.4436285, -0.34766313, 0.061945...",de30fba5d4db4be30007c8119dd8451f
4,AMZN,2010-01-29,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nWe operate cus...,"[0.06934562, -0.21145937, -0.36523664, 0.01908...",de30fba5d4db4be30007c8119dd8451f


In [55]:
test_df.drop(columns=['filing_date'], inplace=True)
test_df.head()

,ticker,year_of_filing,company_name,context,embedding,filing_id
0,AMZN,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nThis Annual Re...,"[0.064206064, -0.54433095, -0.36170813, 0.0565...",de30fba5d4db4be30007c8119dd8451f
1,AMZN,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nWe completed o...,"[0.19773945, -0.6633003, -0.32106644, -0.04229...",de30fba5d4db4be30007c8119dd8451f
2,AMZN,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nWe have organi...,"[0.18689841, -0.24920216, -0.38556015, 0.06467...",de30fba5d4db4be30007c8119dd8451f
3,AMZN,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nWe also manufa...,"[0.14794555, -0.4436285, -0.34766313, 0.061945...",de30fba5d4db4be30007c8119dd8451f
4,AMZN,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nWe operate cus...,"[0.06934562, -0.21145937, -0.36523664, 0.01908...",de30fba5d4db4be30007c8119dd8451f


In [58]:
test_df.columns

Index(['ticker', 'year_of_filing', 'company_name', 'context', 'embedding',
       'filing_id'],
      dtype='object')

In [59]:
test_df_2 = test_df[['year_of_filing', 'company_name', 'context', 'embedding',
       'filing_id', 'ticker']].copy()

In [66]:
merge_df = pd.concat([test_df, test_df_2])
merge_df.head()

,ticker,year_of_filing,company_name,context,embedding,filing_id
0,AMZN,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nThis Annual Re...,"[0.064206064, -0.54433095, -0.36170813, 0.0565...",de30fba5d4db4be30007c8119dd8451f
1,AMZN,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nWe completed o...,"[0.19773945, -0.6633003, -0.32106644, -0.04229...",de30fba5d4db4be30007c8119dd8451f
2,AMZN,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nWe have organi...,"[0.18689841, -0.24920216, -0.38556015, 0.06467...",de30fba5d4db4be30007c8119dd8451f
3,AMZN,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nWe also manufa...,"[0.14794555, -0.4436285, -0.34766313, 0.061945...",de30fba5d4db4be30007c8119dd8451f
4,AMZN,2010,AMAZON COM INC,Company: AMAZON COM INC (AMZN)\nWe operate cus...,"[0.06934562, -0.21145937, -0.36523664, 0.01908...",de30fba5d4db4be30007c8119dd8451f


In [6]:
df[df['ticker'] == 'AAPL']['year_of_filing'].unique()

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
       2021, 2022])

In [17]:
ticker_years = df.groupby('ticker')['year_of_filing'].unique()
ticker_years


ticker
AAPL    [2010, 2011, 2012, 2013, 2014, 2015, 2016, 201...
AMZN    [2010, 2011, 2012, 2013, 2014, 2015, 2016, 201...
GOOG           [2016, 2017, 2018, 2019, 2020, 2021, 2022]
META    [2013, 2014, 2015, 2016, 2017, 2018, 2019, 202...
MSFT    [2010, 2011, 2012, 2013, 2014, 2015, 2016, 201...
NFLX    [2010, 2011, 2012, 2013, 2014, 2015, 2016, 201...
Name: year_of_filing, dtype: object

In [26]:
df.groupby('ticker')['filing_date'].unique()

ticker
AAPL    [2010-10-27, 2011-10-26, 2012-10-31, 2013-10-3...
AMZN    [2010-01-29, 2011-01-28, 2012-02-01, 2013-01-3...
GOOG    [2016-02-11, 2017-02-03, 2018-02-06, 2019-02-0...
META    [2013-02-01, 2014-01-31, 2015-01-29, 2016-01-2...
MSFT    [2010-07-30, 2011-07-28, 2012-07-26, 2013-07-3...
NFLX    [2010-02-22, 2011-02-18, 2012-02-10, 2013-02-0...
Name: filing_date, dtype: object

In [18]:
ty = df.groupby('ticker')['year_of_filing'].nunique()
ty

ticker
AAPL    13
AMZN    13
GOOG     7
META    10
MSFT    13
NFLX    13
Name: year_of_filing, dtype: int64

In [39]:
import pandas as pd
a = pd.read_parquet('df_texts.parquet')
a.head()

,texts,context,embedding,filing_id,company_name,ticker,year_of_filing
0,UNITED STATES SECURITIES AND EXCHANGE COMMISSI...,Company: Apple Inc. (AAPL)\nUNITED STATES SECU...,"[-0.18820103, -0.20937623, -0.34488305, -0.258...",96618433ccb8db908723fb09aa081a34,Apple Inc.,AAPL,2020
1,Yes ☒ No ☐ Indicate by check mark whether the ...,Company: Apple Inc. (AAPL)\nYes ☒ No ☐ Indicat...,"[-0.28359035, -0.44796526, -0.33753213, -0.184...",96618433ccb8db908723fb09aa081a34,Apple Inc.,AAPL,2020
2,Yes ☐ No ☒ The aggregate market value of the v...,Company: Apple Inc. (AAPL)\nYes ☐ No ☒ The agg...,"[-0.2085979, -0.42188534, -0.25574657, 0.14031...",96618433ccb8db908723fb09aa081a34,Apple Inc.,AAPL,2020
3,The 2021 Proxy Statement will be ﬁled with the...,Company: Apple Inc. (AAPL)\nThe 2021 Proxy Sta...,"[-0.037240673, -0.42476544, -0.35393596, -0.36...",96618433ccb8db908723fb09aa081a34,Apple Inc.,AAPL,2020
4,Many of the forward-looking statements are loc...,Company: Apple Inc. (AAPL)\nMany of the forwar...,"[0.06492454, -0.42745784, -0.35366333, -0.2782...",96618433ccb8db908723fb09aa081a34,Apple Inc.,AAPL,2020


In [25]:
import aiohttp
import asyncio
import pdfkit
import re
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from user_agent import generate_user_agent
from pdfminer.high_level import extract_text
import requests
import json
import re
import pdfkit
import json
import sys
import time
import pandas as pd
from user_agent import generate_user_agent
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from tqdm import tqdm
tqdm.pandas()

special_header = {":authority": "www.sec.gov",
":method": "GET",
":path": "/Archives/edgar/data/320193/000119312515356351/d17062d10k.htm",
":scheme": "https",
"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
"accept-encoding": "gzip, deflate, br",
"accept-language": "en-US,en;q=0.9",
"cache-control": "max-age=0",
"sec-ch-ua": '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
"sec-ch-ua-mobile": "?0",
"sec-ch-ua-platform": "macOS",
"sec-fetch-dest": "document",
"sec-fetch-mode": "navigate",
"sec-fetch-site": "none",
"sec-fetch-user": "?1",
"upgrade-insecure-requests": "1",
"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"}


mymodel = '/Users/slidarey/Downloads/rbc-demo/90000'

async def content_embedding(content, model_name):
    '''
    generating embedding for model
    '''
    #Load the model
    # model = SentenceTransformer('sentence-transformers/' + model_name)
    model = SentenceTransformer(model_name)

    embeddings = await asyncio.to_thread(model.encode, content)
    return embeddings


def create_df(list_of_texts):
    '''
    Function to create dataframe from the series of list of dictionaries.
    '''
    master_list = list(list_of_texts)

    df = pd.DataFrame(master_list, columns=['texts'])
    return df


def split_into_sentences(content, no_sntcs=4):
    '''
    This function splits the text into sentences and then groups them into desired number of sentences.
    :param content: text to be split
    :param no_sntcs: number of sentences to be grouped
    :param split_overlap: if True, then the sentences will be split with overlap
    :param no_of_tokens: number of tokens to be grouped
    :param overlap: number of sentences to be overlapped
    :return: list of dictionaries with grouped sentences
    '''
    sentences = re.split('\.\s([A-Z][\w\d]+)', content)
    clnd_sntc = list()
    grpd_sntc = list()

    # splitting the whole text into sentences
    for i in range(len(sentences)):
        if i == 0:
            # first sentence
            clnd_sntc.append(sentences[i].strip() + '.')
        elif (i+2 == len(sentences)):

            # for last sentence = appending captured split pattern to it's sentence
            clnd_sntc.append((sentences[i] + sentences[i+1]).strip())
        elif ((i % 2) != 0):

            # for middle sentences - appending captured split pattern to it's sentence
            clnd_sntc.append(sentences[i] + sentences[i+1] + '.')
        else:
            pass

    # check if number of sentences is more than the desired number of sentences for grouping
    if len(clnd_sntc) >= no_sntcs:

        # to know how many group of sentences can be formed
        for i in range(len(clnd_sntc)//no_sntcs):

            # group and append desired set of sentences
            header = ' '.join(clnd_sntc[(i*no_sntcs):((i+1)*no_sntcs)])
            grpd_sntc.append(header)

            # check if remaining length is less than the desired number of group of sentences
            if (((len(clnd_sntc) - ((i+1)*no_sntcs)) < no_sntcs) and ((len(clnd_sntc) - ((i+1) * no_sntcs)) != 0)):

                # append remaining sentences
                header = ' '.join(clnd_sntc[((i+1)*no_sntcs):])
                grpd_sntc.append(header)
            else:
                pass
    else:
        # return all sentence as 1 group because total number of sentence is not up to desired number of group of sentences
        header = ' '.join(clnd_sntc)
        grpd_sntc.append(header)

    return grpd_sntc




async def get_filing(filing_ticker, filing_year):
    # Generate a user agent string for Chrome
    user_agent = generate_user_agent(navigator='chrome')

    lookup_headers = {
        "referer": "https://sec-api.io",
        "user-agent": user_agent
    }

    html_without_tables = str()
    filing_ticker = str(filing_ticker)
    filing_year = int(filing_year)
    file_link = str()
    company_name = str()
    company_ticker = str()
    filing_id = str()
    data = {
            "query": {"query_string": {
                                        "query": "formType:\"10-K\" AND ticker:(" + filing_ticker + ") AND filedAt:[" + str(
                                        filing_year) + "-01-01 TO " + str(filing_year+1) + "-01-01] AND documentFormatFiles.type:*21*"
                                        }
                    }, "sort": [{
                                    "filedAt": {"order": "desc"}
                                    }]}

    try:
        async with aiohttp.ClientSession() as session:
            async with session.post("https://api.sec-api.io/", headers=lookup_headers, json=data, timeout=10) as response:
                if response.status == 200:
                    parsed_json = await response.json()
                    print(parsed_json)
                    hits = parsed_json['total']['value']
                    if hits > 0:
                        company_name = parsed_json['filings'][0]['companyName']
                        company_ticker = parsed_json['filings'][0]['ticker']
                        examine_link = parsed_json['filings'][0]['linkToFilingDetails']
                        filing_id = parsed_json['filings'][0]['id']
                        if examine_link.find('ix?doc=/') != -1:
                            file_link = examine_link.replace('ix?doc=/', '')
                        else:
                            file_link = examine_link
                    else:
                        print('No hits')
                else:
                    print(response.status)
    except Exception as e:
        print(e)

    # getting the html content of the 10-K filing and removing any image content and table content
    html_to_pdf_retries = 0
    get_page_headers = {"user-agent": user_agent}

    try:
        async with aiohttp.ClientSession() as session:
            print(file_link)
            async with session.get(file_link, headers=special_header, timeout=10) as response:
                if response.status == 200:
                    print('Okay')
                    html = await response.text()
                    html_without_images = re.sub(r'<img.*?>', '', html)
                    html_without_tables = re.sub(r'<table.*?>.*?</table>', '', html_without_images)
                    print(response.status)
                else:
                    print('Not Okay')
                    print(response.status)
    except aiohttp.ClientError as e:
        print(e)
        
    pdfkit.from_string(html_without_tables, 'output.pdf', options={"enable-local-file-access": ""})

    PDF_read = extract_text('Summary.pdf')
    e_PDF_read = PDF_read.replace('\n\n', ' ').replace('\n', ' ')
    e_PDF_read = re.sub(r'\s+', ' ', e_PDF_read)

    # grouped_sentences = split_into_sentences(concatenated_output)
    grouped_sentences = split_into_sentences(e_PDF_read)
    df_texts = create_df(grouped_sentences)

    def create_context(content):
        '''
        Function to create context from the dataset.
        '''
        context = 'Company: ' + company_name + \
            ' (' + company_ticker + ')' + '\n' + content['texts']
        return context

    # creating context using the company_name and new_text column
    df_texts['context'] = df_texts.apply(create_context, axis=1)
    df_texts.to_parquet('your_input_file.parquet', compression='gzip')

    # creating context using the company_name and new_text column
    print('Debug')

    async def process_chunk(chunk, mymodel):
        chunk['context'] = chunk.apply(create_context, axis=1)
        embeddings = await asyncio.gather(*[content_embedding(row['context'], mymodel) for _, row in chunk.iterrows()])
        chunk['embedding'] = embeddings
        chunk['filing_id'] = filing_id
        chunk['company_name'] = company_name
        chunk['ticker'] = company_ticker
        chunk['year_of_filing'] = filing_year
        return chunk

    chunksize = 1  # Adjust this value based on your available RAM
    df_texts = pd.read_parquet('your_input_file.parquet')  # Replace with your input file

    chunks = [df_texts[i:i + chunksize] for i in range(0, df_texts.shape[0], chunksize)]

    processed_chunks = await asyncio.gather(*[process_chunk(chunk, mymodel) for chunk in chunks])

    result = pd.concat(processed_chunks)
    result.to_parquet('df_texts.parquet', compression='gzip')
    print('Parsing Done')

In [ ]:
await asyncio.gather(get_filing('AAPL', 2015))


In [94]:
l = pd.read_parquet('data_with_filing_id.parquet')
# l = l[(l['ticker'] == 'GOOG')].copy()
# search context column where '19.5' is found
l[l['context'].str.contains(r"19\.5", regex=True)]
l.tail

,ticker,year_of_filing,company_name,context,embedding,filing_id
19523,GOOG,2017,Alphabet Inc.,Company: Alphabet Inc. (GOOG)\nManagement’s Re...,"[-0.011912201, -0.105778925, -0.36491787, -0.1...",1788c52bcef0a80094ba392b7118e606
19524,GOOG,2017,Alphabet Inc.,Company: Alphabet Inc. (GOOG)\nThe effectivene...,"[0.029566426, -0.1531196, -0.34326175, -0.0789...",1788c52bcef0a80094ba392b7118e606
19525,GOOG,2017,Alphabet Inc.,Company: Alphabet Inc. (GOOG)\nThe information...,"[0.22770524, -0.22076845, -0.34347808, -0.0508...",1788c52bcef0a80094ba392b7118e606
19526,GOOG,2017,Alphabet Inc.,Company: Alphabet Inc. (GOOG)\nThe information...,"[0.17889473, -0.1796577, -0.32705638, -0.01151...",1788c52bcef0a80094ba392b7118e606
19527,GOOG,2017,Alphabet Inc.,Company: Alphabet Inc. (GOOG)\nDate: February ...,"[0.36548108, -0.16047549, -0.24902934, -0.1806...",1788c52bcef0a80094ba392b7118e606


In [95]:
l.columns

Index(['ticker', 'year_of_filing', 'company_name', 'context', 'embedding',
       'filing_id'],
      dtype='object')

In [93]:
print(l.loc[19335, 'context'])

Company: Alphabet Inc. (GOOG)
Google other revenues The following table presents our Google other revenues (in millions) for the periods presented: Google other revenues consist primarily of revenues and sales from: 26 Our Google other revenues increased $2,926 million from 2015 to 2016 and increased as a percentage of Google segment revenues. These increases were primarily due to the growth in revenues from Google Play, primarily relating to in-app purchases (revenues which we recognize net of payout to developers), hardware sales, and Google Cloud offerings. Our Google other revenues increased $1,104 million from 2014 to 2015 and increased as a percentage of Google segment revenues. These increases were primarily due to the growth of our sales of digital content products in the Google Play store, primarily apps (revenues which we recognize net of payout to partners).


In [96]:
pd.read_parquet('data.parquet')

,ticker,year_of_filing,company_name,context,embedding,filing_id
0,GOOG,2016,Alphabet Inc.,Company: Alphabet Inc. (GOOG)\n10-K 1 goog10-k...,None,b5c5243ad175f6c2580d720e74e2aed1
1,GOOG,2016,Alphabet Inc.,Company: Alphabet Inc. (GOOG)\nYes ý Yes ¨ No ...,None,b5c5243ad175f6c2580d720e74e2aed1
2,GOOG,2016,Alphabet Inc.,Company: Alphabet Inc. (GOOG)\nGoogle Inc. Yes...,None,b5c5243ad175f6c2580d720e74e2aed1
3,GOOG,2016,Alphabet Inc.,Company: Alphabet Inc. (GOOG)\nYes ý Yes ý No ...,None,b5c5243ad175f6c2580d720e74e2aed1
4,GOOG,2016,Alphabet Inc.,Company: Alphabet Inc. (GOOG)\nGoogle Inc. Lar...,None,b5c5243ad175f6c2580d720e74e2aed1
...,...,...,...,...,...,...
5384,META,2022,"Meta Platforms, Inc.","Company: Meta Platforms, Inc. (META)\nWehner C...",None,3820f32b4f59a0c86e41237afd974d06
5385,META,2022,"Meta Platforms, Inc.","Company: Meta Platforms, Inc. (META)\nWehner D...",None,3820f32b4f59a0c86e41237afd974d06
5386,META,2022,"Meta Platforms, Inc.","Company: Meta Platforms, Inc. (META)\nAndreess...",None,3820f32b4f59a0c86e41237afd974d06
5387,META,2022,"Meta Platforms, Inc.","Company: Meta Platforms, Inc. (META)\nKimmitt ...",None,3820f32b4f59a0c86e41237afd974d06
